In [26]:
import sympy as sp
from sympy import symbols, Eq, Sum, Symbol, IndexedBase, solve, Matrix, Eq, Idx

# Most General Kernel

The most general system is given by
$$
    R(t) = \sum_{m=1}^N c_m R_m(t).
$$
For this system the Jacobian reads
$$
    \mathbf{J} = \begin{pmatrix} \beta (1 - 2 I - R) - \rho & -\beta I c_1 & \dots & -\beta I c_N \\
    \rho & - N/T &  &  \\
     & N/T & -N/T & \\
     & & \ddots & \ddots
    \end{pmatrix}
$$

# General Two Step Erlang Sum

In [27]:
# Define symbols
beta, rho, I, k_1, k_2, T = symbols("beta, rho, I, k_1, k_2, T", real=True)
R = IndexedBase('R', real=True)
N, N_mid = symbols("N, N_mid", integer=True)
m = symbols('m', cls=Idx)

def var_vec(N: int):
    return Matrix([I, *(R[mm] for mm in range(1, N+1))])

The most general case for a two step kernel is
$$
    R(t) = \sum_{m=1}^{N} c_m R_m(t) = k_1 \sum_{m=1}^{N_\mathrm{mid}}R_m(t) + k_2 \sum_{m=N_\mathrm{mid}+1}^{N}R_m(t)
$$

## Fixpoint

The FP is given as the solution of
$$\begin{aligned}
    0 &= \beta I (1 - I - R) - \rho I \\
    0 &= \rho I - \frac{N}{T} R_1 \\
    0 &= \frac{N}{T} (R_{m+1} - R_m).
\end{aligned}$$
Therefore,
$$
    R_m = \frac{T\rho}{N} I, \quad\qquad m = 1,\dots, N
$$
and with that
$$
    R = \frac{T\rho}{N} I \left[ (k_1 - k_2) N_\mathrm{mid} + k_2 N \right]
$$

In [10]:
eq = Eq(beta * I * (1 - I - I * T * rho / N * ((k_1 - k_2) * N_mid + k_2 * N)) - rho * I, 0)
fp = solve(eq, I)[1]
fp

N*(beta - rho)/(beta*(N*T*k_2*rho + N + N_mid*T*k_1*rho - N_mid*T*k_2*rho))

## Jacobian

In [29]:
sp.diff(beta * I * (1 - I - sp.Sum(R[m], (m, 1, N))) - rho * I, R[2])

-I*beta*Sum(KroneckerDelta(2, m), (m, 1, N))

In [24]:
def generate_flow(N: int):
    RR = k_1 * Sum(R[m], (m, 1, N_mid)) + k_2 * Sum(R[m], (m, N_mid+1, N))
    flow = [beta * I * (1 - I - RR) - rho * I,
            rho * I - N/T * R[1]]
    for mm in range(2, N+1):
        flow += [(R[mm-1] - R[mm]) * N/T]
    return Matrix(flow)

J = generate_flow(10).jacobian(var_vec(10))
s = {R[mm]: T * rho / 10 * fp for mm in range(1, 10+1)}
s.update({I: fp})
J.subs(s)

Matrix([
[-N*(beta - rho)/(N*T*k_2*rho + N + N_mid*T*k_1*rho - N_mid*T*k_2*rho) + beta*(-N*(beta - rho)/(beta*(N*T*k_2*rho + N + N_mid*T*k_1*rho - N_mid*T*k_2*rho)) - k_1*Sum(R[m], (m, 1, N_mid)) - k_2*Sum(R[m], (m, N_mid + 1, 10)) + 1) - rho, N*(beta - rho)*(-k_1*Piecewise((1, N_mid >= 1), (0, True)) - k_2*Piecewise((1, N_mid <= 0), (0, True)))/(N*T*k_2*rho + N + N_mid*T*k_1*rho - N_mid*T*k_2*rho), N*(beta - rho)*(-k_1*Piecewise((1, N_mid >= 2), (0, True)) - k_2*Piecewise((1, N_mid <= 1), (0, True)))/(N*T*k_2*rho + N + N_mid*T*k_1*rho - N_mid*T*k_2*rho), N*(beta - rho)*(-k_1*Piecewise((1, N_mid >= 3), (0, True)) - k_2*Piecewise((1, N_mid <= 2), (0, True)))/(N*T*k_2*rho + N + N_mid*T*k_1*rho - N_mid*T*k_2*rho), N*(beta - rho)*(-k_1*Piecewise((1, N_mid >= 4), (0, True)) - k_2*Piecewise((1, N_mid <= 3), (0, True)))/(N*T*k_2*rho + N + N_mid*T*k_1*rho - N_mid*T*k_2*rho), N*(beta - rho)*(-k_1*Piecewise((1, N_mid >= 5), (0, True)) - k_2*Piecewise((1, N_mid <= 4), (0, True)))/(N*T*k_2*rho + N

# Block Kernel

In [42]:
# Define variables
I = Symbol('I', real=True)
R = IndexedBase('R', real=True)
c = IndexedBase('c', real=True)
beta = Symbol("beta", real=True, positive=True)
rho = Symbol("rho", real=True, positive=True)
T = Symbol('T', real=True, positive=True)
m = symbols('m', cls=Idx)

def generate_flow(N: int):
    flow = [beta * I * (1 - I - Sum(R[m], (m, 1, N))).doit() - rho * I, # I'
            rho * I - N/T * R[1]]                                       # R_1'
    for mm in range(2, N+1):
        flow += [(R[mm-1] - R[mm]) * N/T]                               # R_m'
    return Matrix(flow)

def var_vec(N: int):
    return Matrix([I, *(R[mm] for mm in range(1, N+1))])

## Fixpoints

In [43]:
def flow_to_eq(flow: Matrix):
    return [Eq(f, 0) for f in flow]

def find_fixpoints(N: int):
    sys = flow_to_eq(generate_flow(N))
    return solve(sys, var_vec(N), dict=True)


N = 5
display(find_fixpoints(N))

[{I: 0, R[1]: 0, R[2]: 0, R[3]: 0, R[4]: 0, R[5]: 0},
 {I: (beta - rho)/(beta*(T*rho + 1)),
  R[1]: T*rho*(beta - rho)/(5*beta*(T*rho + 1)),
  R[2]: T*rho*(beta - rho)/(5*beta*(T*rho + 1)),
  R[3]: T*rho*(beta - rho)/(5*beta*(T*rho + 1)),
  R[4]: T*rho*(beta - rho)/(5*beta*(T*rho + 1)),
  R[5]: T*rho*(beta - rho)/(5*beta*(T*rho + 1))}]

## Jacobian

In [50]:
N = 3
J = generate_flow(N).jacobian(var_vec(N))
sp.simplify(J.subs(find_fixpoints(N)[1]))

Matrix([
[(-beta + rho)/(T*rho + 1), (-beta + rho)/(T*rho + 1), (-beta + rho)/(T*rho + 1), (-beta + rho)/(T*rho + 1)],
[                      rho,                      -3/T,                         0,                         0],
[                        0,                       3/T,                      -3/T,                         0],
[                        0,                         0,                       3/T,                      -3/T]])

# Tests

In [19]:
# Define variables
I = Symbol('I', real=True)
R = IndexedBase('R', real=True)
beta = Symbol("beta", real=True, positive=True)
rho = Symbol("rho", real=True, positive=True)
T = Symbol('T', real=True, positive=True)
m = symbols('m', cls=Idx)

def generate_flow(N: int):
    _R = Sum(R[m], (m, 1, N/2-1)).doit() + 0.5 * Sum(R[m], (m, N/2, N)).doit()
    flow = [beta * I * (1 - I - _R) - rho * I, # I'
            rho * I - N/T * R[1]]                                       # R_1'
    for mm in range(2, N+1):
        flow += [(R[mm-1] - R[mm]) * N/T]                               # R_m'
    return Matrix(flow)

def var_vec(N: int):
    return Matrix([I, *(R[mm] for mm in range(1, N+1))])

def flow_to_eq(flow: Matrix):
    return [Eq(f, 0) for f in flow]

def find_fixpoints(N: int):
    sys = flow_to_eq(generate_flow(N))
    return solve(sys, var_vec(N), dict=True)

generate_flow(10).jacobian(var_vec(10))

Matrix([
[-I*beta + beta*(-I - Sum(R[m], (m, 1, 4.0)) - 0.5*Sum(R[m], (m, 5.0, 10)) + 1) - rho, I*beta*(-Sum(KroneckerDelta(1, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(1, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(2, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(2, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(3, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(3, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(4, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(4, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(5, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(5, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(6, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(6, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(7, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(7, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(8, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(8, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(9, m), (m, 1, 4.0)) - 0.5*Sum(KroneckerDelta(9, m), (m, 5.0, 10))), I*beta*(-Sum(KroneckerDelta(10, m), (m, 